# Exploratory Data Analysis of Genomic Datasets with ADAM and Mango

## Configuring ADAM and Mango on EMR

Mango uses docker containers to be run easily on EMR. To get everything setup and installed, follow EMR documentation at http://bdg-mango.readthedocs.io/en/latest/cloud/emr.html.

# Loading Data from the 1000 Genomes Project

In this tutorial, we will use ADAM and Mango to discover interesting variants in the child of a 1000 Genomes trio.

First, let’s import ADAM and Mango modules, as well as any Spark modules we need:

In [ ]:
# Import ADAM modules
from bdgenomics.adam.adamContext import ADAMContext
from bdgenomics.adam.rdd import AlignmentRecordRDD, CoverageRDD
from bdgenomics.adam.stringency import LENIENT, _toJava

# Import Mango modules
from bdgenomics.mango.alignments import *
from bdgenomics.mango.coverage import CoverageDistribution

# Import Spark modules
from pyspark.sql import functions as sf

Next, we will create an ADAMContext. ADAMContext allows us to load and manipulate genomic data.

In [ ]:
# Create ADAM Context
ac = ADAMContext(spark)

## Variant Analysis with Spark SQL

In this analysis, we will view a trio (NA19685, NA19661, and NA19660) and search for variants that are present in the child but not present in the parents. These are interesting regions, as they may indicate sights of de novo variation that may contribute to multiple disorders.

First, we will load in a subset of variant data from chromosome 17:

In [ ]:
pathPrefix = 's3://1000genomes/phase1/analysis_results/integrated_call_sets/'

genotypesPath = pathPrefix + 'ALL.chr17.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.vcf.gz'
genotypes = ac.loadGenotypes(genotypesPath)

genotypes_df  = genotypes.toDF()

We can take a look at the schema by printing the columns in the dataframe.

In [ ]:
# cache genotypes and show the schema
genotypes_df.columns

This genotypes dataset contains all samples from the 1000 Genomes Project. Therefore, we will next filter genotypes to only consider samples that are in the NA19685 trio, and cache the results in memory.

In [ ]:
# trio IDs
IDs = ['NA19685','NA19661','NA19660']

In [ ]:
# Filter by individuals in the trio
# Note: Loading vcf.gz file takes a long time, as it needs to be collected to the master node first.
# For better performance, you can stage vcf files in s3 or hdfs in blocked gzip format (http://www.htslib.org/doc/bgzip.html)
trio_df = genotypes_df.filter(genotypes_df["sampleId"].isin(IDs))

trio_df.cache()
trio_df.count()

We will next add a new column to our dataframe that determines the genomic location of each variant. This is defined by the chromosome (contigName) and the start and end position of the variant.

In [ ]:
# Add ReferenceRegion column and group by referenceRegion
trios_with_referenceRegion = trio_df.withColumn('ReferenceRegion', 
                    sf.concat(sf.col('contigName'),sf.lit(':'), sf.col('start'), sf.lit('-'), sf.col('end')))

Now, we want to query our dataset to find de novo variants. But first, we must register our dataframe with Spark SQL.

In [ ]:
#  Register df with Spark SQL
trios_with_referenceRegion.createOrReplaceTempView("trios")

Now that our dataframe is registered, we can run SQL queries on it. For our first query, we will select the names of variants belonging to sample NA19685 that have at least one alternative (ALT) allele.

In [ ]:
# filter by alleles. This is a list of variant names that have an alternate allele for the child
alternate_variant_sites = spark.sql("SELECT variant.names[0] AS snp FROM trios \
                                    WHERE array_contains(alleles, 'ALT') AND sampleId == 'NA19685'") 

collected_sites = list(map(lambda x: x.snp, alternate_variant_sites.collect()))

For our next query, we will filter sites in which the parents have both reference alleles. We then filter these variants by the set produced above from the child.

In [ ]:
# get parent records and filter by only REF locations for variant names that were found in the child with an ALT
filtered1 = spark.sql("SELECT * FROM trios WHERE sampleId == 'NA19661' or sampleId == 'NA19660' \
            AND !array_contains(alleles, 'ALT')")


filtered2 = filtered1.filter(filtered1["variant.names"][0].isin(collected_sites))

snp_counts = filtered2.groupBy("variant.names").count().collect()

In [ ]:
# collect snp names as a list
snp_names = map(lambda x: x.names, snp_counts)
denovo_snps = [item for sublist in snp_names for item in sublist]
denovo_snps

Now that we have found some interesting variants, we can unpersist our genotypes from memory.

In [ ]:
trio_df.unpersist()

## Working with Alignment Data

Now, we can explore these specific variant sites in the raw genomic alignment data. First, let’s load in the data for the NA19685 trio:

In [ ]:
# load in NA19685 exome from s3a
childReadsPath = 's3a://1000genomes/phase1/data/NA19685/exome_alignment/NA19685.mapped.illumina.mosaik.MXL.exome.20110411.bam'
parent1ReadsPath = 's3a://1000genomes/phase1/data/NA19660/exome_alignment/NA19660.mapped.illumina.mosaik.MXL.exome.20110411.bam'
parent2ReadsPath = 's3a://1000genomes/phase1/data/NA19661/exome_alignment/NA19661.mapped.illumina.mosaik.MXL.exome.20110411.bam'

childReads = ac.loadAlignments(childReadsPath, stringency=LENIENT)
parent1Reads = ac.loadAlignments(parent1ReadsPath, stringency=LENIENT)
parent2Reads = ac.loadAlignments(parent2ReadsPath, stringency=LENIENT)

We now have data alignment data for three individiuals in our trio. However, the data has not yet been loaded into memory. To cache these datasets for fast subsequent access to our data, we will run the cache() function.

In [ ]:
# cache child RDD
# takes about 2 minutes, on 4 c3.4xlarge worker nodes 
childReads.cache()
childReads.toDF().count()

### Quality Control of Alignment Data

One popular analysis to visually re-affirm the quality of genomic alignment data is by viewing coverage distribution. Coverage distribution gives us an idea of the read coverage we have across a sample. Next, we will generate a sample coverage distribution plot for the child alignment data on chromosome 17.

In [ ]:
# calculate read coverage
# Takes 2-3 minutes
childCoverage = childReads.transform(lambda x: x.filter(x.contigName == "17")).toCoverage()

childCoverage.cache()
childCoverage.toDF().count()

Now that coverage data is calculated and cached, we will compute the coverage distribution of all three samples and plot the coverage distribution.

In [ ]:
# Calculate coverage distribution
# You can check the progress in the SparkUI by navigating to 
# <PUBLIC_MASTER_DNS>:8088 and clicking on the currently running Spark application.
cd = CoverageDistribution(spark, childCoverage, bin_size = 1)

In [ ]:
ax, results = cd.plotDistributions(normalize=True, cumulative=False)

ax.set_title("Coverage Distribution")
ax.set_ylabel("Counts")
ax.set_xlabel("Coverage Depth")
ax.set_xscale("log")
plt.show()

Now that we are done with coverage, we can unpersist these datasets to clear space in memory for the next analysis.

In [ ]:
childCoverage.unpersist()

## Viewing Sites with Missense Variants in the Proband

After verifying alignment data and filtering variants, we have 4 genes with potential missense mutations in the proband, including YBX2, ZNF286B, KSR1, and GNA13. We can visually verify these sites by filtering and viewing the raw reads of the child and parents.

First, let's view the child reads. If we zoom in to the location of the GNA13 variant (63052580-63052581) we can see a heterozygous T to A call.

In [ ]:
# view missense variant at GNA13: 63052580-63052581 (SNP rs201316886) in child
# Takes about 2 minutes to collect data from workers
childViz = AlignmentSummary(spark, ac, childReads)
contig = "17"
start = 63052180
end = 63052981

childViz.viewPileup(contig, start, end)

It looks like there indeed is a variant at this position, possibly a heterozygous SNP with alternate allele A. Let's look at the parent data to verify this variant does not appear in the parents.

In [ ]:
# view missense variant at GNA13: 63052580-63052581 in parent 1
parent1Viz = AlignmentSummary(spark, ac, parent1Reads)
contig = "17"
start = 63052180
end = 63052981

parent1Viz.viewPileup(contig, start, end)

In [ ]:
# view missense variant at GNA13: 63052580-63052581 in parent 2 
parent2Viz = AlignmentSummary(spark, ac, parent2Reads)
contig = "17"
start = 63052180
end = 63052981

parent2Viz.viewPileup(contig, start, end)

This confirms our filter that this variant is indeed only present in the proband, but not the parents.

## Summary

To summarize, this post demonstrated how to setup and run ADAM and Mango in EMR. We demonstrated how to use these tools in an interactive notebook environment to explore the 1000 Genomes Dataset, a publicly available dataset on Amazon S3. We used these tools inspect 1000 Genomes data quality, query for interesting variants in the genome and validate results through visualization of raw datsets.